# Fedbiomed Researcher

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

## Setting the client up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the client
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Connected with result code 0`. it means you are online.

## Create an experiment to train a model on the data found

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [2]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

Note : write **only** the code to export in the following cell

In [3]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self):
        super(MyTrainingPlan, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
               "from torch.utils.data import DataLoader"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


Writing /user/scansiz/home/Desktop/Inria/development/fedbiomed/var/tmp/tmpdxdfkwvf/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the client side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the client side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [4]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of client ID with `clients`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `client_selection_strategy` between the rounds

In [5]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 #clients=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 client_selection_strategy=None)

2021-11-05 15:29:30,777 fedbiomed INFO - Messaging researcher_a7ef3e75-8c26-4c79-a5c7-33a4341aee53 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7fe3c2756c10>
2021-11-05 15:29:30,809 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2021-11-05 15:29:30,813 fedbiomed INFO - log from: node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f - DEBUG Message received: {'researcher_id': 'researcher_a7ef3e75-8c26-4c79-a5c7-33a4341aee53', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2021-11-05 15:29:40,950 fedbiomed DEBUG - torchnn saved model filename: /user/scansiz/home/Desktop/Inria/development/fedbiomed/var/tmpy1ylx9nr/my_model_32342ac8-6e2b-490e-9bbb-49df6140b985.py


Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the clients

In [6]:
exp.run()

2021-11-05 15:29:41,165 fedbiomed INFO - Sampled clients in round 0 ['node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f']
2021-11-05 15:29:41,166 fedbiomed INFO - Send message to client node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f - {'researcher_id': 'researcher_a7ef3e75-8c26-4c79-a5c7-33a4341aee53', 'job_id': '6905dc21-851e-42ef-9465-91b01a5dfced', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2021/11/05/my_model_32342ac8-6e2b-490e-9bbb-49df6140b985.py', 'params_url': 'http://localhost:8844/media/uploads/2021/11/05/my_model_b2a03fd0-d203-4699-b749-8d0d2070e9f9.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f': ['dataset_01c25388-8476-4d65-940d-80c9fc4ee6f2']}}
2021-11-05 15:29:41,166 fedbiomed DEBUG - researcher_a7ef3e75-8c26-4c79-a5c7-33a4341aee53
2021-11-05 15:29:41,172 fedbiomed INFO - log from: node_6

2021-11-05 15:29:58,360 fedbiomed INFO - log from: node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f - INFO Train Epoch: 1 [1440/60000 (2%)]	Loss: 0.192488
2021-11-05 15:29:58,871 fedbiomed INFO - log from: node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f - INFO Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.251375
2021-11-05 15:29:59,355 fedbiomed INFO - log from: node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f - INFO Train Epoch: 1 [2400/60000 (4%)]	Loss: 0.224261
2021-11-05 15:29:59,860 fedbiomed INFO - log from: node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f - INFO Train Epoch: 1 [2880/60000 (5%)]	Loss: 0.133545
2021-11-05 15:30:00,382 fedbiomed INFO - log from: node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f - INFO Train Epoch: 1 [3360/60000 (6%)]	Loss: 0.080605
2021-11-05 15:30:00,890 fedbiomed INFO - log from: node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f - INFO Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.137897
2021-11-05 15:30:01,384 fedbiomed INFO - log from: node_60850d4e-72f5-4dc9-b9aa-3e2f3c31952f - INFO Train Epoch:

Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the clients for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe

Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())


## Optional : searching the data

In [ ]:
from fedbiomed.researcher.requests import Requests

r = Requests()
data = r.search(tags)

import pandas as pd
for node_id in data.keys():
    print('\n','Data for ', node_id, '\n\n', pd.DataFrame(data[node_id]))

## Optional : clean file repository (do not run unless necessary)
Clean all the files in the repo via the rest API.

In [ ]:
# import requests
# from fedbiomed.researcher.environ import environ

# uploaded_models = requests.get(environ['UPLOADS_URL']).json()
# for m in uploaded_models:
#   requests.delete(m['url'])

Feel free to try your own models :D